In [1]:
import pyspark
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession


spark = SparkSession.builder \
    .master("yarn") \
    .appName("GenericAppName") \
    .getOrCreate()

#Access SparkContext from your SparkSession
print("APP Name :"+ spark.sparkContext.appName);
print("Master :"+ spark.sparkContext.master);

APP Name :PySparkShell
Master :yarn


In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
# These are all the databases per year we will analyze.
files = ["players_15", "players_16", "players_17", "players_18",
         "players_19", "players_20", "players_21", "players_22"]
cloud_url = "gs://dataproc-staging-us-east1-1077619237540-pilbvfmf/archive/"
# Reading the .csv files and adding year column for each corresponding database year.
df = spark.read.csv(cloud_url+files[0]+".csv", header=True, inferSchema= True).withColumn("year", lit(2015))
for i in range(1,len(files)):
    df = df.union(spark.read.csv(cloud_url+files[i]+".csv", header=True, inferSchema= True).withColumn("year", lit(2015+i)))

# Adding id column with unique id for every entry. 
# This makes it so the dataframe has only one partition, added to avoid very large numbers for the ids.
df = df.withColumn("id", monotonically_increasing_id())

df_read = df
# I decided to add the columns before ingesting into postgres so they algo get ingested.

In [3]:
# db_properties={}
# #update your db username
# db_properties['username']="postgres"
# #update your db password
# db_properties['password']="bigdata"
# #make sure you got the right port number here
# db_properties['url']= "jdbc:postgresql://localhost:5432/postgres"
# #make sure you had the Postgres JAR file in the right location
# db_properties['driver']="org.postgresql.Driver"
# # Using table in fifa schema.
# db_properties['table']= "fifa.Players"

# # Data ingestion into postgres table
# df.write.format("jdbc")\
# .mode("overwrite")\
# .option("url", db_properties['url'])\
# .option("dbtable", db_properties['table'])\
# .option("user", db_properties['username'])\
# .option("password", db_properties['password'])\
# .option("Driver", db_properties['driver'])\
# .save()


In [4]:
# # Reading database back from postgres.
# df_read = sqlContext.read.format("jdbc")\
#     .option("url", db_properties['url'])\
#     .option("dbtable", db_properties['table'])\
#     .option("user", db_properties['username'])\
#     .option("password", db_properties['password'])\
#     .option("Driver", db_properties['driver'])\
#     .load()


In [5]:
# Table schema
df_read.printSchema()

root
 |-- sofifa_id: integer (nullable = true)
 |-- player_url: string (nullable = true)
 |-- short_name: string (nullable = true)
 |-- long_name: string (nullable = true)
 |-- player_positions: string (nullable = true)
 |-- overall: integer (nullable = true)
 |-- potential: integer (nullable = true)
 |-- value_eur: double (nullable = true)
 |-- wage_eur: double (nullable = true)
 |-- age: integer (nullable = true)
 |-- dob: string (nullable = true)
 |-- height_cm: integer (nullable = true)
 |-- weight_kg: integer (nullable = true)
 |-- club_team_id: double (nullable = true)
 |-- club_name: string (nullable = true)
 |-- league_name: string (nullable = true)
 |-- league_level: integer (nullable = true)
 |-- club_position: string (nullable = true)
 |-- club_jersey_number: integer (nullable = true)
 |-- club_loaned_from: string (nullable = true)
 |-- club_joined: string (nullable = true)
 |-- club_contract_valid_until: integer (nullable = true)
 |-- nationality_id: integer (nullable = tru

In [6]:
# Shows 5 entries.
df_read.show(5, vertical=True)

22/11/29 07:38:12 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


-RECORD 0-------------------------------------------
 sofifa_id                   | 158023               
 player_url                  | https://sofifa.co... 
 short_name                  | L. Messi             
 long_name                   | Lionel Andrés Mes... 
 player_positions            | CF                   
 overall                     | 93                   
 potential                   | 95                   
 value_eur                   | 1.005E8              
 wage_eur                    | 550000.0             
 age                         | 27                   
 dob                         | 1987-06-24           
 height_cm                   | 169                  
 weight_kg                   | 67                   
 club_team_id                | 241.0                
 club_name                   | FC Barcelona         
 league_name                 | Spain Primera Div... 
 league_level                | 1                    
 club_position               | CF             

In [7]:
def player_contracts(df, x):
    # Function prints the x clubs that have the most players with contracts ending in 2023
    df = df.filter((col("year") == 2022) & (col("club_contract_valid_until") == 2023))\
    .groupBy('club_name').count().sort(col('count').desc())
    df.withColumnRenamed('club_name', 'club').withColumnRenamed('count', 'contracts ending 2023').show(x)


In [8]:
player_contracts(df_read, 7)

+--------------------+---------------------+
|                club|contracts ending 2023|
+--------------------+---------------------+
|En Avant de Guingamp|                   19|
| Club Atlético Lanús|                   17|
|       Lechia Gdańsk|                   17|
|        Kasimpaşa SK|                   16|
|        Bengaluru FC|                   16|
|            Barnsley|                   16|
|              Al Tai|                   15|
+--------------------+---------------------+
only showing top 7 rows



In [9]:
def players_over_27(df, y):
    # Function prints the y clubs that have the most average players aged over 27.
    df = df.filter((col("age") > 27) & (col('club_name') != "")).groupBy('club_name', 'year').count()\
        .groupBy('club_name').avg('count').sort(col('avg(count)').desc()) # Calculates the average over the years.
    df.withColumnRenamed('club_name', 'club').withColumnRenamed('avg(count)', 'average players over 27') \
    .show(y)

In [10]:
players_over_27(df_read, 6)

+--------------------+-----------------------+
|                club|average players over 27|
+--------------------+-----------------------+
|  Dorados de Sinaloa|                   19.0|
| Matsumoto Yamaga FC|                   19.0|
| Shanghai Shenhua FC|                   18.5|
|          Qingdao FC|                   18.0|
|Club Deportivo Jo...|                   17.5|
|         Guaireña FC|                   17.0|
+--------------------+-----------------------+
only showing top 6 rows



In [11]:
def most_frequent_position(df):
    # Function prints the most frequent position across national teams every year.
    years = df.select('year').distinct().collect()
    position = []
    for i in range(len(years)):
        position_count = df.groupBy('nation_position','year').count().filter(col("year") == years[i][0]).sort(col('count').desc())
        position.append(position_count.head(2)[1]) # Got the second result as the most common value is null for every year.
    out = spark.createDataFrame(position, schema=position_count.schema)
    out.show()

In [12]:
# Test the function. It shows the most frequent position is 'null' since data cleaning hasn't been carried out yet.
most_frequent_position(df_read)

+---------------+----+-----+
|nation_position|year|count|
+---------------+----+-----+
|            SUB|2015|  564|
|            SUB|2018|  600|
|            SUB|2019|  576|
|            SUB|2022|  396|
|            SUB|2016|  511|
|            SUB|2021|  588|
|            SUB|2017|  564|
|            SUB|2020|  588|
+---------------+----+-----+



## Task III

In [13]:
from pyspark.ml import Pipeline,Transformer
from pyspark.ml.feature import Imputer,StandardScaler,StringIndexer,OneHotEncoder, VectorAssembler

import numpy as np

col_names = ['sofifa_id', 'player_url', 'short_name', 'long_name', 'player_positions',
             'overall', 'potential', 'value_eur', 'wage_eur', 'age', 'dob', 'height_cm',
             'weight_kg', 'club_team_id', 'club_name', 'league_name', 'league_level',
             'club_position', 'club_jersey_number', 'club_loaned_from', 'club_joined',
             'club_contract_valid_until', 'nationality_id', 'nationality_name', 'nation_team_id',
             'nation_position', 'nation_jersey_number', 'preferred_foot', 'weak_foot',
             'skill_moves', 'international_reputation', 'work_rate', 'body_type', 'real_face',
             'release_clause_eur', 'player_tags', 'player_traits', 'pace', 'shooting', 'passing',
             'dribbling', 'defending', 'physic', 'attacking_crossing', 'attacking_finishing',
             'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys',
             'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing',
             'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed',
             'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power',
             'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
             'mentality_aggression', 'mentality_interceptions', 'mentality_positioning',
             'mentality_vision', 'mentality_penalties', 'mentality_composure',
             'defending_marking_awareness', 'defending_standing_tackle', 'defending_sliding_tackle',
             'goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking',
             'goalkeeping_positioning', 'goalkeeping_reflexes', 'goalkeeping_speed', 'ls',
             'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm',
             'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb',
             'gk', 'player_face_url', 'club_logo_url', 'club_flag_url', 'nation_logo_url',
             'nation_flag_url', 'year', 'id']

nominal_cols = ['league_name', 'club_position','preferred_foot',
                'work_rate', 'body_type', 'nationality_name']

continuous_cols = ['potential', 'value_eur', 'wage_eur', 'age',
       'height_cm', 'weight_kg', 'league_level', 
       'club_contract_valid_until', 'weak_foot', 'skill_moves',
       'international_reputation', 'pace', 'shooting', 'passing', 'dribbling',
       'defending', 'physic', 'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
       'movement_reactions', 'movement_balance', 'power_shot_power',
       'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
       'mentality_aggression', 'mentality_interceptions',
       'mentality_positioning', 'mentality_vision', 'mentality_penalties',
       'mentality_composure', 'defending_marking_awareness',
       'defending_standing_tackle', 'defending_sliding_tackle',
       'goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking',
       'goalkeeping_positioning', 'goalkeeping_reflexes',
       'cf', 'lcm', 'gk']

sum_cols = ['cf', 'lcm', 'gk']
string_cols = ['ls', 'st', 'rs', 'lw', 'lf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'cm',
             'rcm', 'rm', 'lwb', 'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb']
        

cols_to_remove = ['dob', 'player_positions', 'club_joined', 'club_loaned_from', 'nation_team_id',
                  'nation_position', 'nation_jersey_number', 'player_tags', 'player_traits',
                  'goalkeeping_speed', 'nation_logo_url', 'player_url', 'player_face_url',
                  'club_logo_url', 'club_flag_url', 'nation_flag_url', 'sofifa_id', 'short_name',
                  'long_name', 'real_face', 'club_jersey_number', 'club_team_id', 'nationality_id',
                  'club_name', 'release_clause_eur']

class OutcomeCreater(Transformer): # this defines a transformer that creates the outcome column
    
    def __init__(self):
        super().__init__()

    def _transform(self, dataset):
        output_df = dataset.withColumn('outcome', col('overall')).drop("overall")  
        # output_df = output_df.withColumn('outcome', col('outcome').cast(DoubleType()))
        return output_df

def sum_to_int(str):
    out=0
    for i,c in enumerate(str):
        if c == '+':
            out = int(str[:i]) + int(str[i+1:])
    return out


class FeatureTypeCaster(Transformer): # this transformer will cast the columns as appropriate types  
    def __init__(self):
        super().__init__()

    def _transform(self, dataset):
        output_df = dataset

        to_int = udf(lambda x: sum_to_int(x),IntegerType())
        for col_name in sum_cols:
            output_df = output_df.withColumn(col_name,to_int(col(col_name)))
        for col_name in (continuous_cols):
            output_df = output_df.withColumn(col_name,col(col_name).cast(DoubleType()))
        
        # This simplifies the body_type feature to only the 4 body type labels (unique, stocky, lean, normal)
        output_df = output_df.withColumn("body_type", split(output_df.body_type, ' ').getItem(0))
        return output_df

    
class FillNa(Transformer): # this defines a transformer that fills null values with number for imputing
    
    def __init__(self, columns_to_be_imputed = None):
        super().__init__()
        self.columns_to_be_imputed=columns_to_be_imputed
    def _transform(self, dataset):
        output_df = dataset.fillna(-200, self.columns_to_be_imputed)
        return output_df
    

class NaDropper(Transformer): # this defines a transformer that drops rows with null values
    
    def __init__(self, cols):
        self.cols = cols
        super().__init__()
    def _transform(self, dataset):
        output_df = dataset.na.drop(subset = self.cols)
        return output_df
    
    
class ColumnDropper(Transformer): # this transformer drops unnecessary columns
    def __init__(self, columns_to_drop = None):
        super().__init__()
        self.columns_to_drop=columns_to_drop
    def _transform(self, dataset):
        output_df = dataset
        for col_name in self.columns_to_drop:
            output_df = output_df.drop(col_name)
        return output_df

def get_preprocess_pipeline():
    # Stage where columns are casted as appropriate types
    stage_typecaster = FeatureTypeCaster()

    cols_to_be_imputed = ['value_eur', 'wage_eur',
       'club_contract_valid_until', 'pace', 'shooting', 'passing', 'dribbling',
       'defending', 'physic', 'mentality_composure']
#     imputed_cols = [x+"_imputed" for x in cols_to_be_imputed]
    
#     value_not_in_dataset = -200
#     stage_fillna = FillNa(columns_to_be_imputed = cols_to_be_imputed)
#     stage_imputer = Imputer (
#             inputCols=cols_to_be_imputed,
#             outputCols=imputed_cols,
#             strategy = "median",
#             missingValue = value_not_in_dataset)
    
#     stage_column_dropper_1 = ColumnDropper(columns_to_drop = cols_to_remove)
    stage_na_handler1 = NaDropper(nominal_cols)
    
    # Stage where nominal columns are transformed to index columns using StringIndexer
    nominal_id_cols = [x+"_index" for x in nominal_cols]
    nominal_onehot_cols = [x+"_encoded" for x in nominal_cols]
    stage_nominal_indexer = StringIndexer(inputCols = nominal_cols, outputCols = nominal_id_cols )

    # Stage where the index columns are further transformed using OneHotEncoder
    stage_nominal_onehot_encoder = OneHotEncoder(inputCols=nominal_id_cols, outputCols=nominal_onehot_cols)
    

    # Stage where all relevant features are assembled into a vector (and dropping a few)
    feature_cols = (continuous_cols +cols_to_be_imputed + nominal_onehot_cols)
    
    correlated_cols_to_remove = ['defending_marking_awareness', 'defending_standing_tackle', 
                    'defending_sliding_tackle', 'mentality_interceptions', 'movement_acceleration', 
                    'movement_sprint_speed', 'attacking_finishing', 'attacking_volleys', 
                    'power_shot_power', 'power_long_shots', 'mentality_positioning', 
                    'attacking_crossing', 'attacking_short_passing', 'skill_long_passing', 
                    'skill_ball_control', 'power_strength', 'skill_curve', 'goalkeeping_diving', 
                    'goalkeeping_handling', 'goalkeeping_kicking', 'goalkeeping_positioning']
    for col_name in (correlated_cols_to_remove+cols_to_be_imputed):
        feature_cols.remove(col_name)
    
    stage_na_handler2 = NaDropper(feature_cols)

    stage_vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="vectorized_features")

    # Stage where we scale the columns
    stage_scaler = StandardScaler(inputCol= 'vectorized_features', outputCol= 'features')
    

    # Stage for creating the outcome column representing whether there is attack 
    stage_outcome = OutcomeCreater()

    # Removing all unnecessary columns, only keeping the 'features' and 'outcome' columns
    stage_column_dropper = ColumnDropper(columns_to_drop = cols_to_remove+nominal_cols+nominal_id_cols+
        nominal_onehot_cols + cols_to_be_imputed + continuous_cols+string_cols+['vectorized_features', 'year', 'id'])
    # Connect the columns into a pipeline
    pipeline = Pipeline(stages=[stage_typecaster, stage_na_handler1,
                                stage_nominal_indexer,stage_nominal_onehot_encoder,
        stage_na_handler2, stage_vector_assembler,stage_scaler,stage_outcome,stage_column_dropper])
    return pipeline 

In [33]:
preprocess_pipeline = get_preprocess_pipeline()
preprocess_pipeline_model = preprocess_pipeline.fit(df_read)

fifa_df = preprocess_pipeline_model.transform(df_read)

# Creating the training, validation and test data sets

fifa_df_train, fifa_df_validate, fifa_df_test = fifa_df.randomSplit([0.6, 0.2, 0.2], 14813)
fifa_df.unpersist()

DataFrame[features: vector, outcome: int]

## Spark ML: Linear Regression and Random Forest Regression

In [15]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

evaluator = RegressionEvaluator(
    labelCol="outcome", predictionCol="prediction", metricName="mse")

In [16]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol = 'features', labelCol = 'outcome', predictionCol = 'prediction',
                      maxIter = 100, regParam = 0.05)
lr_model = lr.fit(fifa_df_train)



22/11/29 07:39:09 WARN com.github.fommil.netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/11/29 07:39:09 WARN com.github.fommil.netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
22/11/29 07:39:09 WARN com.github.fommil.netlib.LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
22/11/29 07:39:09 WARN com.github.fommil.netlib.LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [17]:
lr_prediction_train = lr_model.transform(fifa_df_train)
lr_prediction_test = lr_model.transform(fifa_df_test)

lr_mse_train = evaluator.evaluate(lr_prediction_train)
lr_mse_test = evaluator.evaluate(lr_prediction_test)


print(f"Train MSE = {np.round(lr_mse_train,5)}, test MSE = {np.round(lr_mse_test,5)}")


Train MSE = 2.74408, test MSE = 2.80226


In [18]:
lr_paramGrid = (ParamGridBuilder()
             .addGrid(lr.maxIter, [100, 200, 500])# maximum number of iterations
             .addGrid(lr.regParam,[0.0001 ,0.05, 0.1])# Regularization parameter
             .build())

lr_cv = CrossValidator(estimator=lr, estimatorParamMaps=lr_paramGrid, 
                    evaluator=evaluator, numFolds=5, parallelism = 2)

lr_cv_model = lr_cv.fit(fifa_df_train)

lr_cv_prediction_test = lr_cv_model.transform(fifa_df_test)
lr_cv_mse = evaluator.evaluate(lr_cv_prediction_test)


In [19]:
print(f"Before cross-validation and parameter tuning, MSE={np.round(lr_mse_test,5)}")
print(f"After cross-validation and parameter tuning, MSE={np.round(lr_cv_mse,5)}")

Before cross-validation and parameter tuning, MSE=2.80226
After cross-validation and parameter tuning, MSE=2.77996


In [20]:
from pyspark.ml.regression import RandomForestRegressor

rf = RandomForestRegressor(featuresCol = 'features', labelCol = 'outcome')
rf_model = rf.fit(fifa_df_train)

In [21]:
rf_prediction_train = rf_model.transform(fifa_df_train)
rf_prediction_test = rf_model.transform(fifa_df_test)

rf_mse_train = evaluator.evaluate(rf_prediction_train)
rf_mse_test = evaluator.evaluate(rf_prediction_test)


print(f"Train MSE = {np.round(rf_mse_train,5)}, test MSE = {np.round(rf_mse_test,5)}")


Train MSE = 2.29817, test MSE = 2.34569


In [22]:
rf_paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [3, 10])# maximum depth for each tree
             .addGrid(rf.minInfoGain,[0.0, 1.0, 5.0])# number of trees
             .build())

rf_cv = CrossValidator(estimator=rf, estimatorParamMaps=rf_paramGrid, 
                    evaluator=evaluator, numFolds=5, parallelism = 2)

rf_cv_model = rf_cv.fit(fifa_df_train)

rf_cv_prediction_test = rf_cv_model.transform(fifa_df_test)
rf_cv_mse = evaluator.evaluate(rf_cv_prediction_test)


22/11/29 07:42:20 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 1328.8 KiB
22/11/29 07:42:23 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 2.2 MiB
22/11/29 07:42:28 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.8 MiB
22/11/29 07:42:31 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 1004.6 KiB
22/11/29 07:42:35 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 6.6 MiB
22/11/29 07:43:11 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 1329.0 KiB
22/11/29 07:43:14 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 2.2 MiB
22/11/29 07:43:18 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.8 MiB
22/11/29 07:43:23 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task

In [23]:
print(f"Before cross-validation and parameter tuning, MSE={np.round(rf_mse_test,5)}")
print(f"After cross-validation and parameter tuning, MSE={np.round(rf_cv_mse,5)}")

Before cross-validation and parameter tuning, MSE=2.34569
After cross-validation and parameter tuning, MSE=0.86258


# Tensorflow Shallow and deep neural network regression

In [34]:
to_array = udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))

fifa_train = fifa_df_train.withColumn('features', to_array('features'))
fifa_train.cache()
fifa_df_train_pandas = fifa_train.toPandas()
fifa_train.unpersist()

fifa_validate = fifa_df_validate.withColumn('features', to_array('features'))
fifa_validate.cache()
fifa_df_validate_pandas = fifa_validate.toPandas()
fifa_validate.unpersist()

fifa_test = fifa_df_test.withColumn('features', to_array('features'))
fifa_test.cache()
fifa_df_test_pandas = fifa_test.toPandas()
fifa_test.unpersist()

DataFrame[features: array<float>, outcome: int]

In [35]:
import tensorflow as tf
from tensorflow import keras 

# Converting the pandas DataFrame to tensors

x_train = tf.constant(np.array(fifa_df_train_pandas['features'].values.tolist()))
y_train = tf.constant(np.array(fifa_df_train_pandas['outcome'].values.tolist()))

x_validate = tf.constant(np.array(fifa_df_validate_pandas['features'].values.tolist()))
y_validate = tf.constant(np.array(fifa_df_validate_pandas['outcome'].values.tolist()))


x_test = tf.constant(np.array(fifa_df_test_pandas['features'].values.tolist()))
y_test = tf.constant(np.array(fifa_df_test_pandas['outcome'].values.tolist()))


In [36]:
lr_model = keras.Sequential( [
                            keras.layers.Dense(1)
                            ] )


y_pred = lr_model(x_train)
lr_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (57651, 1)                313       
Total params: 313
Trainable params: 313
Non-trainable params: 0
_________________________________________________________________


In [27]:
# Compile the model
lr_model.compile(optimizer = 'adam',
    loss=tf.keras.losses.MeanSquaredError(), 
    metrics=[tf.keras.metrics.MeanSquaredError()])

In [28]:
lr_model.fit(x_train,y_train, epochs = 10,validation_data=(x_validate,y_validate),verbose = 2)

2022-11-29 07:47:04.274266: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-11-29 07:47:04.274799: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2249995000 Hz


Epoch 1/10
1815/1815 - 1s - loss: 95.7520 - mean_squared_error: 95.7520 - val_loss: 12.8874 - val_mean_squared_error: 12.8874
Epoch 2/10
1815/1815 - 1s - loss: 9.7342 - mean_squared_error: 9.7342 - val_loss: 8.0368 - val_mean_squared_error: 8.0368
Epoch 3/10
1815/1815 - 1s - loss: 6.9905 - mean_squared_error: 6.9905 - val_loss: 6.6772 - val_mean_squared_error: 6.6772
Epoch 4/10
1815/1815 - 1s - loss: 5.7853 - mean_squared_error: 5.7853 - val_loss: 5.2226 - val_mean_squared_error: 5.2226
Epoch 5/10
1815/1815 - 1s - loss: 5.1223 - mean_squared_error: 5.1223 - val_loss: 5.4002 - val_mean_squared_error: 5.4002
Epoch 6/10
1815/1815 - 1s - loss: 4.6934 - mean_squared_error: 4.6934 - val_loss: 4.8075 - val_mean_squared_error: 4.8075
Epoch 7/10
1815/1815 - 1s - loss: 4.4038 - mean_squared_error: 4.4038 - val_loss: 4.1777 - val_mean_squared_error: 4.1777
Epoch 8/10
1815/1815 - 1s - loss: 4.2117 - mean_squared_error: 4.2117 - val_loss: 5.0473 - val_mean_squared_error: 5.0473
Epoch 9/10
1815/1815

In [29]:
from tensorboard.plugins.hparams import api as hp

# Defining the hyperparameters that will be compared and tuned using tensorboard

HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([0.001,0.005,0.01])) # the learning rate

# Creating the logs
with tf.summary.create_file_writer('logs/lr_model').as_default():
    hp.hparams_config(
        hparams=[HP_LEARNING_RATE],
        metrics=[hp.Metric('MSE')],
    )



In [30]:
# Defining the model training function for hyperparameter tuning
def train_test_model(hparams,logdir, x, y, i, k):

    model = keras.Sequential()
    model.add(keras.layers.Dense(1))
    model.compile(
        optimizer = tf.optimizers.Adam(learning_rate=hparams[HP_LEARNING_RATE]),
        loss = tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.MeanSquaredError(name = 'MSE_Epochs')])
    
  # This section is for splitting the data sets into k folds. May be better to do this outside this function
    x_train = tf.split(x, k)
    y_train = tf.split(y, k)
    val_x = x_train.pop(i)
    val_y = y_train.pop(i)
    x_train = tf.concat([x_train[0], x_train[1]], 0)  
    y_train = tf.concat([y_train[0], y_train[1]], 0)  

  # Store the output of the model training
    history = model.fit(x_train, y_train, epochs=20, verbose = 2,
    callbacks=[tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)],
    validation_data = (val_x, val_y))

    mse = np.min(history.history["val_MSE_Epochs"]) # Returns the lowest Mean Squared Error obtained in training
    return mse


In [37]:
# Combine the original training and validation data sets
# x_train = tf.concat([x_train,x_validate], 0)
# y_train = tf.concat([y_train,y_validate], 0)

# Get randomized indices in order to shuffle data sets
indices = tf.range(start=0, limit=tf.shape(x_train)[0], dtype=tf.int32)

shuffled_indices = tf.random.shuffle(indices)

# Shuffle the data sets with the random indices
x_train_shuffled = tf.gather(x_train, shuffled_indices)
y_train_shuffled = tf.gather(y_train, shuffled_indices)
print(tf.shape(x_train_shuffled))

# Number of folds to split the data into
k = 3

tf.Tensor([57651   312], shape=(2,), dtype=int32)


In [38]:
# Nested loops for hyperparameter tuning, iterating over all the hyperparameters and combinations
for hp_lr in HP_LEARNING_RATE.domain.values:
    hparams = {
        HP_LEARNING_RATE: hp_lr,
    }
    run_name = f"run-LEARNING_RATE {(hparams[HP_LEARNING_RATE])}"
    print('--- Starting trial: %s' % run_name)
    print({h.name: hparams[h] for h in hparams})

    run_dir = 'logs/lr_model/' + run_name
        
    # The model is trained with each of the k folds in the dataset and mse is averaged over the k runs.
    mse = 0
    for i in range(k):
        mse += train_test_model(hparams, run_dir, x_train_shuffled, y_train_shuffled, i, k)
    mse = mse / k

    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        tf.summary.scalar("MSE", mse, step=1)


--- Starting trial: run-LEARNING_RATE 0.001
{'learning_rate': 0.001}
Epoch 1/20


2022-11-29 07:50:17.225969: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-11-29 07:50:17.226025: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-11-29 07:50:17.226468: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2022-11-29 07:50:17.472571: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-11-29 07:50:17.472614: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-11-29 07:50:17.474993: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-11-29 07:50:17.475580: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2022-11-29 07:50:17.476511: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/lr_model/run-LEARNING_RATE 0.001/train/plugins/profile/2022_11_29_07_50_17
2022-11-29 07:50:17.47

1202/1202 - 1s - loss: 134.1344 - MSE_Epochs: 134.1344 - val_loss: 28.0743 - val_MSE_Epochs: 28.0743
Epoch 2/20
1202/1202 - 1s - loss: 18.5208 - MSE_Epochs: 18.5208 - val_loss: 13.6232 - val_MSE_Epochs: 13.6232
Epoch 3/20
1202/1202 - 1s - loss: 11.1852 - MSE_Epochs: 11.1852 - val_loss: 9.7477 - val_MSE_Epochs: 9.7477
Epoch 4/20
1202/1202 - 1s - loss: 8.5029 - MSE_Epochs: 8.5029 - val_loss: 7.5881 - val_MSE_Epochs: 7.5881
Epoch 5/20
1202/1202 - 1s - loss: 6.8401 - MSE_Epochs: 6.8401 - val_loss: 6.2889 - val_MSE_Epochs: 6.2889
Epoch 6/20
1202/1202 - 1s - loss: 5.8695 - MSE_Epochs: 5.8695 - val_loss: 5.5378 - val_MSE_Epochs: 5.5378
Epoch 7/20
1202/1202 - 1s - loss: 5.3049 - MSE_Epochs: 5.3049 - val_loss: 5.6745 - val_MSE_Epochs: 5.6745
Epoch 8/20
1202/1202 - 1s - loss: 4.9326 - MSE_Epochs: 4.9326 - val_loss: 4.8697 - val_MSE_Epochs: 4.8697
Epoch 9/20
1202/1202 - 1s - loss: 4.6658 - MSE_Epochs: 4.6658 - val_loss: 5.1227 - val_MSE_Epochs: 5.1227
Epoch 10/20
1202/1202 - 1s - loss: 4.4669 - M

2022-11-29 07:50:35.366341: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-11-29 07:50:35.366396: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-11-29 07:50:35.366821: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2022-11-29 07:50:35.600790: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-11-29 07:50:35.600831: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-11-29 07:50:35.612900: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-11-29 07:50:35.613447: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2022-11-29 07:50:35.614218: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/lr_model/run-LEARNING_RATE 0.001/train/plugins/profile/2022_11_29_07_50_35
2022-11-29 07:50:35.61

1202/1202 - 1s - loss: 96.6836 - MSE_Epochs: 96.6836 - val_loss: 16.9467 - val_MSE_Epochs: 16.9467
Epoch 2/20
1202/1202 - 1s - loss: 12.0514 - MSE_Epochs: 12.0514 - val_loss: 9.2506 - val_MSE_Epochs: 9.2506
Epoch 3/20
1202/1202 - 1s - loss: 8.0297 - MSE_Epochs: 8.0297 - val_loss: 7.9259 - val_MSE_Epochs: 7.9259
Epoch 4/20
1202/1202 - 1s - loss: 6.6849 - MSE_Epochs: 6.6849 - val_loss: 6.3346 - val_MSE_Epochs: 6.3346
Epoch 5/20
1202/1202 - 1s - loss: 5.8721 - MSE_Epochs: 5.8721 - val_loss: 5.8444 - val_MSE_Epochs: 5.8444
Epoch 6/20
1202/1202 - 1s - loss: 5.3781 - MSE_Epochs: 5.3781 - val_loss: 5.3123 - val_MSE_Epochs: 5.3123
Epoch 7/20
1202/1202 - 1s - loss: 5.0030 - MSE_Epochs: 5.0030 - val_loss: 5.2266 - val_MSE_Epochs: 5.2266
Epoch 8/20
1202/1202 - 1s - loss: 4.7179 - MSE_Epochs: 4.7179 - val_loss: 4.7474 - val_MSE_Epochs: 4.7474
Epoch 9/20
1202/1202 - 1s - loss: 4.5055 - MSE_Epochs: 4.5055 - val_loss: 4.6967 - val_MSE_Epochs: 4.6967
Epoch 10/20
1202/1202 - 1s - loss: 4.3536 - MSE_Epo

...

1202/1202 - 1s - loss: 19.2776 - MSE_Epochs: 19.2776 - val_loss: 6.4598 - val_MSE_Epochs: 6.4598
Epoch 2/20
1202/1202 - 1s - loss: 6.7476 - MSE_Epochs: 6.7476 - val_loss: 6.1900 - val_MSE_Epochs: 6.1900
Epoch 3/20
1202/1202 - 1s - loss: 6.0412 - MSE_Epochs: 6.0412 - val_loss: 26.9927 - val_MSE_Epochs: 26.9927
Epoch 4/20
1202/1202 - 1s - loss: 5.5366 - MSE_Epochs: 5.5366 - val_loss: 4.9253 - val_MSE_Epochs: 4.9253
Epoch 5/20
1202/1202 - 1s - loss: 5.5055 - MSE_Epochs: 5.5055 - val_loss: 5.3662 - val_MSE_Epochs: 5.3662
Epoch 6/20
1202/1202 - 1s - loss: 5.2149 - MSE_Epochs: 5.2149 - val_loss: 16.7906 - val_MSE_Epochs: 16.7906
Epoch 7/20
1202/1202 - 1s - loss: 4.8983 - MSE_Epochs: 4.8983 - val_loss: 4.4020 - val_MSE_Epochs: 4.4020
Epoch 8/20
1202/1202 - 1s - loss: 4.9457 - MSE_Epochs: 4.9457 - val_loss: 17.9234 - val_MSE_Epochs: 17.9234
Epoch 9/20
1202/1202 - 1s - loss: 4.8839 - MSE_Epochs: 4.8839 - val_loss: 4.5165 - val_MSE_Epochs: 4.5165
Epoch 10/20
1202/1202 - 1s - loss: 4.8431 - MSE_E

2022-11-29 07:52:30.515473: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-11-29 07:52:30.515524: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-11-29 07:52:30.515885: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2022-11-29 07:52:30.749227: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-11-29 07:52:30.749269: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-11-29 07:52:30.761593: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-11-29 07:52:30.762156: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2022-11-29 07:52:30.762986: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/lr_model/run-LEARNING_RATE 0.01/train/plugins/profile/2022_11_29_07_52_30
2022-11-29 07:52:30.763

1202/1202 - 1s - loss: 9.8596 - MSE_Epochs: 9.8596 - val_loss: 6.5671 - val_MSE_Epochs: 6.5671
Epoch 2/20
1202/1202 - 1s - loss: 6.2356 - MSE_Epochs: 6.2356 - val_loss: 6.1375 - val_MSE_Epochs: 6.1375
Epoch 3/20
1202/1202 - 1s - loss: 5.9143 - MSE_Epochs: 5.9143 - val_loss: 16.3830 - val_MSE_Epochs: 16.3830
Epoch 4/20
1202/1202 - 1s - loss: 5.4518 - MSE_Epochs: 5.4518 - val_loss: 4.6265 - val_MSE_Epochs: 4.6265
Epoch 5/20
1202/1202 - 1s - loss: 5.1364 - MSE_Epochs: 5.1364 - val_loss: 7.0145 - val_MSE_Epochs: 7.0145
Epoch 6/20
1202/1202 - 1s - loss: 5.2435 - MSE_Epochs: 5.2435 - val_loss: 3.9968 - val_MSE_Epochs: 3.9968
Epoch 7/20
1202/1202 - 1s - loss: 4.8878 - MSE_Epochs: 4.8878 - val_loss: 3.7941 - val_MSE_Epochs: 3.7941
Epoch 8/20
1202/1202 - 1s - loss: 4.9614 - MSE_Epochs: 4.9614 - val_loss: 7.3092 - val_MSE_Epochs: 7.3092
Epoch 9/20
1202/1202 - 1s - loss: 4.9119 - MSE_Epochs: 4.9119 - val_loss: 10.4279 - val_MSE_Epochs: 10.4279
Epoch 10/20
1202/1202 - 1s - loss: 5.0955 - MSE_Epoch

2022-11-29 07:52:48.714425: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-11-29 07:52:48.714476: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-11-29 07:52:48.714524: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2022-11-29 07:52:48.956779: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-11-29 07:52:48.956824: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-11-29 07:52:48.970473: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-11-29 07:52:48.971050: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2022-11-29 07:52:48.971860: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/lr_model/run-LEARNING_RATE 0.01/train/plugins/profile/2022_11_29_07_52_48
2022-11-29 07:52:48.972

1202/1202 - 1s - loss: 152.8249 - MSE_Epochs: 152.8249 - val_loss: 7.8452 - val_MSE_Epochs: 7.8452
Epoch 2/20
1202/1202 - 1s - loss: 7.0992 - MSE_Epochs: 7.0992 - val_loss: 6.2467 - val_MSE_Epochs: 6.2467
Epoch 3/20
1202/1202 - 1s - loss: 5.8499 - MSE_Epochs: 5.8499 - val_loss: 5.2256 - val_MSE_Epochs: 5.2256
Epoch 4/20
1202/1202 - 1s - loss: 5.5340 - MSE_Epochs: 5.5340 - val_loss: 10.5508 - val_MSE_Epochs: 10.5508
Epoch 5/20
1202/1202 - 1s - loss: 5.6774 - MSE_Epochs: 5.6774 - val_loss: 4.5731 - val_MSE_Epochs: 4.5731
Epoch 6/20
1202/1202 - 1s - loss: 5.4994 - MSE_Epochs: 5.4994 - val_loss: 14.4499 - val_MSE_Epochs: 14.4499
Epoch 7/20
1202/1202 - 1s - loss: 5.2506 - MSE_Epochs: 5.2506 - val_loss: 6.9334 - val_MSE_Epochs: 6.9334
Epoch 8/20
1202/1202 - 1s - loss: 5.1703 - MSE_Epochs: 5.1703 - val_loss: 11.8683 - val_MSE_Epochs: 11.8683
Epoch 9/20
1202/1202 - 1s - loss: 4.9374 - MSE_Epochs: 4.9374 - val_loss: 3.9610 - val_MSE_Epochs: 3.9610
Epoch 10/20
1202/1202 - 1s - loss: 4.6384 - MSE

In [39]:
# Defining the model with 5 hidden layers and the output layer for 5 classes

deep_model = keras.Sequential( [keras.layers.Dense(20,activation='relu'),
                                keras.layers.Dense(20,activation='relu'),
                                keras.layers.Dense(20,activation='relu'),
                                keras.layers.Dense(20,activation='relu') ,
                                keras.layers.Dense(1)] ) # Added softmax activation, output will be probability


y_pred = deep_model(x_train)
deep_model.summary()


Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (57651, 20)               6260      
_________________________________________________________________
dense_13 (Dense)             (57651, 20)               420       
_________________________________________________________________
dense_14 (Dense)             (57651, 20)               420       
_________________________________________________________________
dense_15 (Dense)             (57651, 20)               420       
_________________________________________________________________
dense_16 (Dense)             (57651, 1)                21        
Total params: 7,541
Trainable params: 7,541
Non-trainable params: 0
_________________________________________________________________


In [40]:
# Compile the model
deep_model.compile(optimizer = 'adam',
    loss=tf.keras.losses.MeanSquaredError(), 
    metrics=[tf.keras.metrics.MeanSquaredError()])

In [41]:
deep_model.fit(x_train,y_train, epochs = 10,validation_data=(x_validate,y_validate),verbose = 2)

Epoch 1/10
1802/1802 - 2s - loss: 32.5639 - mean_squared_error: 32.5639 - val_loss: 5.1771 - val_mean_squared_error: 5.1771
Epoch 2/10
1802/1802 - 2s - loss: 4.6433 - mean_squared_error: 4.6433 - val_loss: 3.8984 - val_mean_squared_error: 3.8984
Epoch 3/10
1802/1802 - 2s - loss: 3.8322 - mean_squared_error: 3.8322 - val_loss: 3.9156 - val_mean_squared_error: 3.9156
Epoch 4/10
1802/1802 - 2s - loss: 3.2149 - mean_squared_error: 3.2149 - val_loss: 2.6890 - val_mean_squared_error: 2.6890
Epoch 5/10
1802/1802 - 2s - loss: 2.8567 - mean_squared_error: 2.8567 - val_loss: 2.2981 - val_mean_squared_error: 2.2981
Epoch 6/10
1802/1802 - 2s - loss: 2.7310 - mean_squared_error: 2.7310 - val_loss: 2.2821 - val_mean_squared_error: 2.2821
Epoch 7/10
1802/1802 - 2s - loss: 2.5254 - mean_squared_error: 2.5254 - val_loss: 2.7918 - val_mean_squared_error: 2.7918
Epoch 8/10
1802/1802 - 2s - loss: 2.4506 - mean_squared_error: 2.4506 - val_loss: 2.6441 - val_mean_squared_error: 2.6441
Epoch 9/10
1802/1802 -

In [42]:
HP_WIDTH = hp.HParam('NN_width', hp.Discrete([20, 30, 40])) # the width of the layers
HP_DEPTH = hp.HParam('NN_depth', hp.Discrete([4,6])) # Number of layers/depth

# Creating the logs
with tf.summary.create_file_writer('logs/deep_model').as_default():
    hp.hparams_config(
        hparams=[HP_WIDTH, HP_DEPTH],
        metrics=[hp.Metric('MSE')],
    )

In [43]:
# Defining the model training function for hyperparameter tuning
def train_test_deep_model(hparams,logdir, x, y, i, k):
    model = keras.Sequential()
    for _ in range(hparams[HP_DEPTH]): #Create the model
        model.add(keras.layers.Dense(hparams[HP_WIDTH],activation='relu'))
    model.add(keras.layers.Dense(1))
    model.compile(
        optimizer='adam',
        loss = tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.MeanSquaredError(name = 'MSE_Epochs')])
    
  # This section is for splitting the data sets into k folds. May be better to do this outside this function
    x_train = tf.split(x, k)
    y_train = tf.split(y, k)
    val_x = x_train.pop(i)
    val_y = y_train.pop(i)
    x_train = tf.concat([x_train[0], x_train[1]], 0)  
    y_train = tf.concat([y_train[0], y_train[1]], 0)  

  # Store the output of the model training
    history = model.fit(x_train, y_train, epochs=20, verbose = 2,
    callbacks=[tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)],
    validation_data = (val_x, val_y))

    mse = np.min(history.history["val_MSE_Epochs"]) # Returns the lowest Mean Squared Error obtained in training
    return mse


In [44]:
# Nested loops for hyperparameter tuning, iterating over all the hyperparameters and combinations
for hp_width in HP_WIDTH.domain.values:
    for hp_depth in (HP_DEPTH.domain.values):
        hparams = {
            HP_WIDTH: hp_width,
            HP_DEPTH: hp_depth,
        }
        run_name = f"run-WIDTH {int(hparams[HP_WIDTH])}-DEPTH {hparams[HP_DEPTH]}"
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})

        run_dir = 'logs/deep_model/' + run_name
        
        # The model is trained with each of the k folds in the dataset and accuracy is averaged over the k runs.
        mse = 0
        for i in range(k):
            mse += train_test_deep_model(hparams, run_dir, x_train_shuffled, y_train_shuffled, i, k)
        mse = mse / k

        with tf.summary.create_file_writer(run_dir).as_default():
            hp.hparams(hparams)  # record the values used in this trial
            tf.summary.scalar("MSE", mse, step=1)

    

--- Starting trial: run-WIDTH 20-DEPTH 4
{'NN_width': 20, 'NN_depth': 4}
Epoch 1/20


2022-11-29 07:55:56.190104: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-11-29 07:55:56.190165: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-11-29 07:55:56.190572: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2022-11-29 07:55:57.104014: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-11-29 07:55:57.104060: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-11-29 07:55:57.146418: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-11-29 07:55:57.147203: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2022-11-29 07:55:57.148386: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/deep_model/run-WIDTH 20-DEPTH 4/train/plugins/profile/2022_11_29_07_55_57
2022-11-29 07:55:57.149

1202/1202 - 2s - loss: 35.3163 - MSE_Epochs: 35.3163 - val_loss: 5.7583 - val_MSE_Epochs: 5.7583
Epoch 2/20
1202/1202 - 1s - loss: 5.2761 - MSE_Epochs: 5.2761 - val_loss: 11.3831 - val_MSE_Epochs: 11.3831
Epoch 3/20
1202/1202 - 1s - loss: 4.3698 - MSE_Epochs: 4.3698 - val_loss: 4.6808 - val_MSE_Epochs: 4.6808
Epoch 4/20
1202/1202 - 1s - loss: 3.6806 - MSE_Epochs: 3.6806 - val_loss: 4.6444 - val_MSE_Epochs: 4.6444
Epoch 5/20
1202/1202 - 1s - loss: 3.2547 - MSE_Epochs: 3.2547 - val_loss: 3.5825 - val_MSE_Epochs: 3.5825
Epoch 6/20
1202/1202 - 1s - loss: 3.0163 - MSE_Epochs: 3.0163 - val_loss: 2.4254 - val_MSE_Epochs: 2.4254
Epoch 7/20
1202/1202 - 1s - loss: 2.8744 - MSE_Epochs: 2.8744 - val_loss: 2.9124 - val_MSE_Epochs: 2.9124
Epoch 8/20
1202/1202 - 1s - loss: 2.5769 - MSE_Epochs: 2.5769 - val_loss: 2.1160 - val_MSE_Epochs: 2.1160
Epoch 9/20
1202/1202 - 1s - loss: 2.5026 - MSE_Epochs: 2.5026 - val_loss: 3.9267 - val_MSE_Epochs: 3.9267
Epoch 10/20
1202/1202 - 1s - loss: 2.4426 - MSE_Epoch

2022-11-29 07:56:38.057236: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-11-29 07:56:38.057299: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-11-29 07:56:38.057733: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


Epoch 1/20


...

1202/1202 - 2s - loss: 51.7077 - MSE_Epochs: 51.7077 - val_loss: 6.1751 - val_MSE_Epochs: 6.1751
Epoch 2/20
1202/1202 - 2s - loss: 5.1261 - MSE_Epochs: 5.1261 - val_loss: 4.7524 - val_MSE_Epochs: 4.7524
Epoch 3/20
1202/1202 - 2s - loss: 4.2112 - MSE_Epochs: 4.2112 - val_loss: 3.8624 - val_MSE_Epochs: 3.8624
Epoch 4/20
1202/1202 - 2s - loss: 3.6725 - MSE_Epochs: 3.6725 - val_loss: 12.1312 - val_MSE_Epochs: 12.1312
Epoch 5/20
1202/1202 - 2s - loss: 3.5113 - MSE_Epochs: 3.5113 - val_loss: 3.6399 - val_MSE_Epochs: 3.6399
Epoch 6/20
1202/1202 - 2s - loss: 3.0431 - MSE_Epochs: 3.0431 - val_loss: 6.4290 - val_MSE_Epochs: 6.4290
Epoch 7/20
1202/1202 - 2s - loss: 2.8837 - MSE_Epochs: 2.8837 - val_loss: 2.2640 - val_MSE_Epochs: 2.2640
Epoch 8/20
1202/1202 - 2s - loss: 2.7076 - MSE_Epochs: 2.7076 - val_loss: 4.8837 - val_MSE_Epochs: 4.8837
Epoch 9/20
1202/1202 - 2s - loss: 2.5644 - MSE_Epochs: 2.5644 - val_loss: 4.1542 - val_MSE_Epochs: 4.1542
Epoch 10/20
1202/1202 - 2s - loss: 2.4478 - MSE_Epoch

2022-11-29 08:06:36.764817: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-11-29 08:06:36.764874: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-11-29 08:06:36.765244: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


Epoch 1/20


2022-11-29 08:06:37.157436: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-11-29 08:06:37.157480: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-11-29 08:06:37.181299: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-11-29 08:06:37.182187: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2022-11-29 08:06:37.183326: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/deep_model/run-WIDTH 40-DEPTH 6/train/plugins/profile/2022_11_29_08_06_37
2022-11-29 08:06:37.184095: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/deep_model/run-WIDTH 40-DEPTH 6/train/plugins/profile/2022_11_29_08_06_37/cluster-652e-m.trace.json.gz
2022-11-29 08:06:37.185123: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/deep_mo

1202/1202 - 2s - loss: 28.2822 - MSE_Epochs: 28.2822 - val_loss: 8.0937 - val_MSE_Epochs: 8.0937
Epoch 2/20
1202/1202 - 2s - loss: 5.9452 - MSE_Epochs: 5.9452 - val_loss: 4.6597 - val_MSE_Epochs: 4.6597
Epoch 3/20
1202/1202 - 2s - loss: 4.7884 - MSE_Epochs: 4.7884 - val_loss: 3.5906 - val_MSE_Epochs: 3.5906
Epoch 4/20
1202/1202 - 2s - loss: 4.3360 - MSE_Epochs: 4.3360 - val_loss: 3.2626 - val_MSE_Epochs: 3.2626
Epoch 5/20
1202/1202 - 2s - loss: 3.7576 - MSE_Epochs: 3.7576 - val_loss: 4.0075 - val_MSE_Epochs: 4.0075
Epoch 6/20
1202/1202 - 2s - loss: 3.5439 - MSE_Epochs: 3.5439 - val_loss: 7.9830 - val_MSE_Epochs: 7.9830
Epoch 7/20
1202/1202 - 2s - loss: 3.3877 - MSE_Epochs: 3.3877 - val_loss: 8.0163 - val_MSE_Epochs: 8.0163
Epoch 8/20
1202/1202 - 2s - loss: 3.1179 - MSE_Epochs: 3.1179 - val_loss: 2.8766 - val_MSE_Epochs: 2.8766
Epoch 9/20
1202/1202 - 2s - loss: 3.0629 - MSE_Epochs: 3.0629 - val_loss: 2.4616 - val_MSE_Epochs: 2.4616
Epoch 10/20
1202/1202 - 2s - loss: 2.7191 - MSE_Epochs: